# Import libraries and load text

In [2]:
import os
print(os.environ['CONDA_DEFAULT_ENV'])

CFPB


In [6]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize, FreqDist
from nltk.stem.wordnet import WordNetLemmatizer
import string
import plotly
import plotly.graph_objects as go
import plotly.express as px
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [5]:
!python3 -m pip install plotly


Defaulting to user installation because normal site-packages is not writeable
  Using cached plotly-5.9.0-py2.py3-none-any.whl (15.2 MB)
  Using cached tenacity-8.0.1-py3-none-any.whl (24 kB)


In [8]:
# Prepping dataframe as done in 1_EDA.ipynb

df = pd.read_csv('complaints_0721_2018-2022.csv')
df = df[['Product', 'Consumer complaint narrative']]

df = df.rename(columns={"Product": "product", "Consumer complaint narrative": "narrative"})
# df = df[ (df['product'] != "Debt collection" ) & 
#         (df['product'] != "Money transfer, virtual currency, or money service") & 
#        (df['product'] != "Checking or savings account")]


print(df['product'].value_counts())

df['product'].replace({'Credit reporting, credit repair services, or other personal consumer reports': 'Non Money Transfer',
                       'Credit card or prepaid card': 'Non Money Transfer',
                       'Mortgage': 'Non Money Transfer',
                       'Vehicle loan or lease': 'Non Money Transfer',
                       'Payday loan, title loan, or personal loan': 'Non Money Transfer',
                       'Student loan': 'Non Money Transfer',
                       'Debt collection' : 'Non Money Transfer', 
                       'Money transfer, virtual currency, or money service': 'Money Transfer', 
                       'Checking or savings account': 'Non Money Transfer'}, inplace=True)

Credit reporting, credit repair services, or other personal consumer reports    312752
Debt collection                                                                 110530
Credit card or prepaid card                                                      58116
Mortgage                                                                         46866
Checking or savings account                                                      36304
Money transfer, virtual currency, or money service                               18876
Vehicle loan or lease                                                            14295
Student loan                                                                     14220
Payday loan, title loan, or personal loan                                         9574
Name: product, dtype: int64


In [9]:
df['product'].value_counts()

Non Money Transfer    602657
Money Transfer         18876
Name: product, dtype: int64

In [10]:
train_pos = df[df['product'] == 'Money Transfer'].sample(frac=0.15)
df_rest_pos = df[df['product'] == 'Money Transfer'].drop(train_pos.index)

train_neg = df[df['product'] == 'Non Money Transfer'].sample(frac=0.27)
df_rest_neg = df[df['product'] == 'Non Money Transfer'].drop(train_neg.index)


In [11]:
df_rest = pd.concat([df_rest_pos, df_rest_neg])
df_train = pd.concat([train_pos, train_neg])

In [12]:
df_train['product'].value_counts()

Non Money Transfer    162717
Money Transfer          2831
Name: product, dtype: int64

In [13]:
df_rest['product'].value_counts()

Non Money Transfer    439940
Money Transfer         16045
Name: product, dtype: int64

In [14]:
df_train.shape


(165548, 2)

In [15]:
df_train.tail(10)

,product,narrative
428461,Non Money Transfer,I have contacted total visa to let them know I...
568615,Non Money Transfer,"I filled out a survey for XXXX XXXX, where I ..."
190362,Non Money Transfer,"Equifax sent a letter dated XX/XX/2021, it sta..."
499297,Non Money Transfer,"This is XXXX XXXX XXXX, who is submitting this..."
77257,Non Money Transfer,I found out about the debt when trying to refi...
382991,Non Money Transfer,1. XXXX XXXX XXXX Not mine Date reported XX/XX...
235624,Non Money Transfer,XXXX/XXXX XXXX ACCT NO XXXX IS BEING WRONGFULL...
523015,Non Money Transfer,XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX OH XXX...
325324,Non Money Transfer,I have checking account with overdraft protect...
387224,Non Money Transfer,Ftc violations XXXX : These lawsuits allege XX...


In [16]:
df_train.index = np.arange(0, len(df_train))

In [17]:
df_train.tail(10)

,product,narrative
165538,Non Money Transfer,I have contacted total visa to let them know I...
165539,Non Money Transfer,"I filled out a survey for XXXX XXXX, where I ..."
165540,Non Money Transfer,"Equifax sent a letter dated XX/XX/2021, it sta..."
165541,Non Money Transfer,"This is XXXX XXXX XXXX, who is submitting this..."
165542,Non Money Transfer,I found out about the debt when trying to refi...
165543,Non Money Transfer,1. XXXX XXXX XXXX Not mine Date reported XX/XX...
165544,Non Money Transfer,XXXX/XXXX XXXX ACCT NO XXXX IS BEING WRONGFULL...
165545,Non Money Transfer,XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX OH XXX...
165546,Non Money Transfer,I have checking account with overdraft protect...
165547,Non Money Transfer,Ftc violations XXXX : These lawsuits allege XX...


In [18]:
df_train['product'].value_counts()

Non Money Transfer    162717
Money Transfer          2831
Name: product, dtype: int64

In [19]:
df_train.loc[0]['narrative']

"On XX/XX/2019, I used WorldRemit to send money internationally to XXXX WorldRemit 's website was seemingly very clear about all of the fees involved. The transaction went smoothly. However, when I received my credit card bill from XXXX ( statement for billing period XX/XX/19-XX/XX/19 ), I found I was charged a large fee for making a cash advance. \n\nWhen I made the transaction on WorldRemit, there was nothing to indicate that the transaction would be treated as a cash advance. I called XXXX to complain, and they told me that they had no control over the fee -- that the merchant ( WorldRemit ) submitted the transaction as a cash advance. I called WorldRemit to complain, and they told me that they had no control over the way in which the bank processed the transaction -- that they did not submit it as a cash advance, but that XXXX chose to process it as a cash advance. \n\nSince neither company accepts responsibility for the additional fee, I am considering both to be committing fraud.

In [56]:
len(df_train)

165548

# Loop through narratives to remove stopwords, tokenize and lemmatize

In [24]:
stopwords_list = stopwords.words('english') + list(string.punctuation)
stopwords_list += ["''", '""', '...', '``']
stopwords_list += ['--', 'xxxx', 'xx/xx/2020', 'xx/xx/2021', 'xx/xx/2019', 'xx/xx/2018','xx/xx/18', 'xx/xx/19']

In [29]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/keqiaoli/nltk_data...


True

In [26]:
# function to tokenize data and remove stopwords
def process_narrative(narrative):
    tokens = nltk.word_tokenize(narrative)
    stopwords_removed = [token.lower() for token in tokens if token.lower() not in stopwords_list]
    
    # adding line to remove all tokens with numbers and punctuation
    stopwords_punc_and_numbers_removed = [word for word in stopwords_removed if word.isalpha()]
    
    return stopwords_punc_and_numbers_removed


# function to concat words (used in function below)
def concat_words(list_of_words):
    # remove any NaN's
    # list_of_words = [i for i in list if i is not np.nan]

    concat_words = ''
    for word in list_of_words:
        concat_words += word + ' '
    return concat_words.strip()

# function to lemmatize words and merge each complaint into a single space-separated string

lemm = WordNetLemmatizer()

def make_lemma_and_concat(list_of_words):
    # remove any NaN's
    list_of_words = [i for i in list_of_words if i is not np.nan]
    
    # lemmatize each word
    lemmatized_list = []
    for idx, word in enumerate(list_of_words):
        lemmatized_list.append(lemm.lemmatize(word))
    
    # make the list into a single string with the words separated by ' '
    concatenated_string = concat_words(lemmatized_list)
    return concatenated_string

# Prepare dataframe for modeling

In [27]:
df_train['processed narrative'] = df_train['narrative']
df_train.shape

(165548, 3)

In [30]:
for i in range(len(df_train)):
    processed_narr = process_narrative(df_train['narrative'].loc[i])
    narr = make_lemma_and_concat(processed_narr)
    df_train['processed narrative'].loc[i] = narr
    if i % 3000 == 0:
        print(f'Finished line number {i}')
df_train.head()

Finished line number 0
Finished line number 3000
Finished line number 6000
Finished line number 9000
Finished line number 12000
Finished line number 15000
Finished line number 18000
Finished line number 21000
Finished line number 24000
Finished line number 27000
Finished line number 30000
Finished line number 33000
Finished line number 36000
Finished line number 39000
Finished line number 42000
Finished line number 45000
Finished line number 48000
Finished line number 51000
Finished line number 54000
Finished line number 57000
Finished line number 60000
Finished line number 63000
Finished line number 66000
Finished line number 69000
Finished line number 72000
Finished line number 75000
Finished line number 78000
Finished line number 81000
Finished line number 84000
Finished line number 87000
Finished line number 90000
Finished line number 93000
Finished line number 96000
Finished line number 99000
Finished line number 102000
Finished line number 105000
Finished line number 108000
Finis

,product,narrative,processed narrative
0,Money Transfer,"On XX/XX/2019, I used WorldRemit to send money...",used worldremit send money internationally wor...
1,Money Transfer,"On XX/XX/2020, my son had initiated money tran...",son initiated money transfer paypal account pa...
2,Money Transfer,"On Thursday, XX/XX/XXXX, my son sent me {$1000...",thursday son sent via cash app went go cash ap...
3,Money Transfer,This dispute is with Bank of America. It relat...,dispute bank america relates unconscionable co...
4,Money Transfer,western union : - XXXX recording date : - XX/X...,western union recording date amount usd amount...


In [31]:
df_rest.shape

(455985, 2)

In [32]:
df_rest.head()

,product,narrative
44,Money Transfer,"In XXXX, I opened and funded a consumer accoun..."
120,Money Transfer,"Previously, I had repeatedly received spam mai..."
121,Money Transfer,"Hello, I contacted ( cryptocurrency exchange )..."
163,Money Transfer,Retained Paysafe Merchant services on XX/XX/20...
181,Money Transfer,I had made a transfer to the wrong person of 4...


In [33]:
df_rest['processed narrative'] = df_rest['narrative']
df_rest.index = np.arange(0, len(df_rest))
for i in range(len(df_rest)):
    processed_narr = process_narrative(df_rest['narrative'].loc[i])
    narr = make_lemma_and_concat(processed_narr)
    df_rest['processed narrative'].loc[i] = narr
    if i % 3000 == 0:
        print(f'Finished line number {i}')
df_rest.head()

Finished line number 0
Finished line number 3000
Finished line number 6000
Finished line number 9000
Finished line number 12000
Finished line number 15000
Finished line number 18000
Finished line number 21000
Finished line number 24000
Finished line number 27000
Finished line number 30000
Finished line number 33000
Finished line number 36000
Finished line number 39000
Finished line number 42000
Finished line number 45000
Finished line number 48000
Finished line number 51000
Finished line number 54000
Finished line number 57000
Finished line number 60000
Finished line number 63000
Finished line number 66000
Finished line number 69000
Finished line number 72000
Finished line number 75000
Finished line number 78000
Finished line number 81000
Finished line number 84000
Finished line number 87000
Finished line number 90000
Finished line number 93000
Finished line number 96000
Finished line number 99000
Finished line number 102000
Finished line number 105000
Finished line number 108000
Finis

,product,narrative,processed narrative
0,Money Transfer,"In XXXX, I opened and funded a consumer accoun...",opened funded consumer account bam trading ser...
1,Money Transfer,"Previously, I had repeatedly received spam mai...",previously repeatedly received spam mail pnc b...
2,Money Transfer,"Hello, I contacted ( cryptocurrency exchange )...",hello contacted cryptocurrency exchange suppor...
3,Money Transfer,Retained Paysafe Merchant services on XX/XX/20...,retained paysafe merchant service handle payme...
4,Money Transfer,I had made a transfer to the wrong person of 4...,made transfer wrong person contacted cash app ...


# Save dataframe as csv for use in other notebooks

In [34]:
df_train.to_csv('complaints_processed_0722_train.csv')

In [37]:
df_train.head()

,product,narrative,processed narrative
0,Money Transfer,"On XX/XX/2019, I used WorldRemit to send money...",used worldremit send money internationally wor...
1,Money Transfer,"On XX/XX/2020, my son had initiated money tran...",son initiated money transfer paypal account pa...
2,Money Transfer,"On Thursday, XX/XX/XXXX, my son sent me {$1000...",thursday son sent via cash app went go cash ap...
3,Money Transfer,This dispute is with Bank of America. It relat...,dispute bank america relates unconscionable co...
4,Money Transfer,western union : - XXXX recording date : - XX/X...,western union recording date amount usd amount...


In [38]:
df_rest.head()

,product,narrative,processed narrative
0,Money Transfer,"In XXXX, I opened and funded a consumer accoun...",opened funded consumer account bam trading ser...
1,Money Transfer,"Previously, I had repeatedly received spam mai...",previously repeatedly received spam mail pnc b...
2,Money Transfer,"Hello, I contacted ( cryptocurrency exchange )...",hello contacted cryptocurrency exchange suppor...
3,Money Transfer,Retained Paysafe Merchant services on XX/XX/20...,retained paysafe merchant service handle payme...
4,Money Transfer,I had made a transfer to the wrong person of 4...,made transfer wrong person contacted cash app ...


In [39]:
df_rest.to_csv('complaints_processed_0722_test.csv')